*This code is part of the book 'Übungs- und Lernbuch Wahrscheinlichkeitstheorie und Stochastik' by [Dr. Niklas Hebestreit-Düsing](https://dr-hebestreit.de/)* 

**(Aufgabe 34, Konstruktion erzeugter $\sigma$-Algebren).**
Es sei $X$ eine Menge, $n \in \mathbb N$ eine natürliche Zahl und $\mathfrak{E} := \{E_1 , \dotsc, E_n\}$ ein Mengensystem, das aus endlich vielen Teilmengen von $X$ besteht. 
Im Folgenden sollen Sie eine Konstruktionsmethode für die Berechnung der von $\mathfrak{E}$ erzeugten $\sigma$-Algebra auf $X$ nachvollziehen. Gehen Sie dazu wie folgt vor:

(a) (Partition der Menge $X$). Für eine Teilmenge $E$ von $X$ werden $E^0 := E^\mathsf{c}$ und $E^1 := E$ definiert. Weiter wird
$$
E_\alpha := \bigcap_{k=1}^n E^{\alpha_k}
$$
für $\alpha \in \{0,1\}^n$ gesetzt. Weisen Sie nach, dass das System 
$$
\mathfrak{P} := \big\{ E_\alpha \mid \alpha\in \{0,1\}^n \big\} \setminus \{\emptyset\}
$$
eine Partition von $X$ ist.

(b) Es bezeichne $\mathfrak{A}$ das Mengensystem, das aus allen endlichen Vereinigungen von Mengen aus $\mathfrak{P}$ besteht, also
$$
\mathfrak{A} := \left\{ \bigsqcup_{E \, \in \, \mathfrak I} E \mid  \mathfrak I \subseteq \mathfrak{P} \right\}
$$
Beweisen Sie $\sigma(\mathfrak{E}) = \mathfrak{A}$, das heißt, die von $\mathfrak{E}$ erzeugte $\sigma$-Algebra stimmt mit $\mathfrak{A}$ überein. 
	
(c) Bestimmen Sie mithilfe der Konstruktionsmethode aus Teil (b) die vom System $\mathfrak{E} := \{ \{1\}, \{2,3\}\}$ erzeugte $\sigma$-Algebra auf $X := \{1,2,3,4\}$. Bestätigen Sie Ihr Ergebnis mithilfe von $\texttt{SageMath}$.
	
(d) (Implementation des Konstruktionsverfahrens). Implementieren Sie beispielsweise in $\texttt{SageMath}$ die Funktion
$$
\texttt{generate\_sigma\_algebras(X)}
$$
die alle $\sigma$-Algebren über der endlichen Menge $X$ berechnet. Geben Sie anschließend alle $\sigma$-Algebren über $\{1\}$, $\{1,2\}$ und $\{1,2,3\}$ an. 
	
(e) Untersuchen Sie, welches der beiden Mengensystem
$$
\mathfrak{A} := \big\{ \emptyset, \{1\}, \{3\}, \{1,2\}, \{3,4\}, \{1,3,4\}, \{2,3,4\}, \{1,2,3,4\} \big\}
$$
und
$$
\mathfrak{B} := \big\{ \emptyset, \{1\}, \{2\}, \{1,3\}, \{1,3,4\}, \{2,3,4\}, \{1,2,3,4\} \big\}
$$
eine $\sigma$-Algebra über der Menge $\{1,2,3,4\}$ definiert.

**Lösung (c).**

In [5]:
# Partition of the set X = {1, 2, 3, 4}.

P = [[1], [4], [2, 3]]  

# Generate the sigma-algebra generated by the partition 'P'.
# For each subset E in the powerset of 'P', compute the union 
# of the subsets in E. This gives all possible unions of the 
# original partition elements.

sigma_algebra = [list(set().union(*E)) for E in powerset(P)]

# Display the resulting sigma-algebra.
print(sigma_algebra)

[[], [1], [4], [1, 4], [2, 3], [1, 2, 3], [2, 3, 4], [1, 2, 3, 4]]


**Lösung (d).**

In [5]:
def cond_intersection(E:list[list[int]], X:list[int], alpha:list[int]) -> list[int]: 
    """
    Compute a conditional intersection of sets based on a 
    binary vector.

    Arguments:
        E (list): A list of lists.
        X (list): The base set from which the conditional 
                  intersection is computed.
        alpha (list): A binary vector of 0s and 1s indicating 
                      inclusion (1) or exclusion (0) of each 
                      corresponding subset in E.

    Returns:
    intsc (list): The conditional intersection of E with 
                  respect to alpha.
    """
    
    # Start with the full set X.
    intsc = X
    for i in range(len(alpha)):
        if alpha[i] == 1:
            # Include elements in E[i].
            intsc = list(set(intsc) & set(E[i]))
        else:
            # Exclude elements in E[i] by intersecting with the
            # complement of E[i]
            intsc = list(set(intsc) & (set(X) - set(E[i])))
    return intsc

In [7]:
E = [[1], [2, 3]]  
X = [1, 2, 3, 4]
alpha = [0, 0]

cond_intersection(E, X, alpha)

[4]

In [8]:
beta = [0, 1]

cond_intersection(E, X, beta)

[2, 3]

In [9]:
import itertools

def partition(E:list[list[int], X:list[int]) -> list[int]:
    """
    Generates a partition of the universal set 'X' based on a family of subsets 'E'.

    Arguments:
    E (list): A list of lists.
    X (list): The base set.

    Returns:
    P (list) : A partition of 'X' based on 'E'.
    """
    
    P = [] 
    # List of all binary indicator vectors with entries 0 or 1.
    vecs = list(itertools.product([0, 1], repeat = len(E)))  

    for alpha in vecs:
        cond_intsc = cond_intersection(E, X, alpha)
        if cond_intsc:
            # Only keep non-empty subsets.
            P.append(cond_intsc)

    return P

In [10]:
E = [[1], [2, 3]]  
X = [1, 2, 3, 4]

partition(E, X)

[[4], [2, 3], [1]]

In [9]:
def sigma_algebra_from_partition(P:list[list[int]]) -> list[int]:
    """
    Generates a sigma-algebra based on the partition 'P'.
    
    Arguments:
    P (list): A partition of a universal set.
    
    Returns:
    (list): All possible unions of the subsets in 'P'. 
            This forms the sigma-algebra generated by the 
            partition 'P'.
    """ 
    
    return [list(set().union(*E)) for E in powerset(P)]

In [12]:
P = [[1], [2, 3], [4]]

sigma_algebra_from_partition(P)

[[], [1], [2, 3], [1, 2, 3], [4], [1, 4], [2, 3, 4], [1, 2, 3, 4]]

In [13]:
from itertools import combinations

def generate_sigma_algebras(X:list[int]) -> list[list[int]]:
    """
    Generates all sigma-algebras over the finite set 'X' in
    the following way:
    - Loop over all subsets of 'X' and compute the 
      partition generated by it.
    - If a partition is found, compute the sigma-algebra 
      generated by that partition.
    - Collect and return all distinct sigma-algebras.
    
    Arguments:
    X (list): The base set.
    
    Returns:
    (list): A list of sigma-algebras, where each sigma-algebra 
            is a list of lists.
    """
    
    # A list to store all generated sigma-algebras; may 
    # include duplicates.
    sigma_algebras = []

    # Generate all combinations of i subsets from the powerset 
    # of 'X'. Each element in subsys is a list of i subsets from 'X'.
    for i in range(2** len(X) + 1):
        S = [list(A) for A in combinations(powerset(X), i)]
        
        # Generate a partition of 'X' and the 
        # sigma-algebra from that partition.
        for E in S:
            P = partition(E, X)
            alg = sigma_algebra_from_partition(P)
            # Save the sigma-algebra.
            sigma_algebras.append(alg)
            
    result = []
    # Clean up: remove all duplicate sigma-algebras.
    for alg in sigma_algebras:
        alg = sorted([sorted(A) for A in alg])
        if alg not in result:
            result.append(alg)
        
    return result

In [14]:
# Sigma-algebras over the empty set {}
generate_sigma_algebras([])

[[[]]]

In [15]:
# Sigma-algebras over the set {1}
generate_sigma_algebras([1])

[[[], [1]]]

In [16]:
# Sigma-algebras over the set {1, 2}
generate_sigma_algebras([1, 2])

[[[], [1, 2]], [[], [1], [1, 2], [2]]]

In [17]:
# Sigma-algebras over the set {1, 2, 3}
generate_sigma_algebras([1, 2, 3])

[[[], [1, 2, 3]],
 [[], [1], [1, 2, 3], [2, 3]],
 [[], [1, 2, 3], [1, 3], [2]],
 [[], [1, 2], [1, 2, 3], [3]],
 [[], [1], [1, 2], [1, 2, 3], [1, 3], [2], [2, 3], [3]]]

**Lösung (e).**

In [20]:
# Note: this may take some time to run!

# Sigma-algebras over the set {1, 2, 3, 4}
generate_sigma_algebras([1, 2, 3, 4])

[[[], [1, 2, 3, 4]],
 [[], [1], [1, 2, 3, 4], [2, 3, 4]],
 [[], [1, 2, 3, 4], [1, 3, 4], [2]],
 [[], [1, 2], [1, 2, 3, 4], [3, 4]],
 [[], [1, 2, 3, 4], [1, 2, 4], [3]],
 [[], [1, 2, 3, 4], [1, 3], [2, 4]],
 [[], [1, 2, 3, 4], [1, 4], [2, 3]],
 [[], [1, 2, 3], [1, 2, 3, 4], [4]],
 [[], [1], [1, 2], [1, 2, 3, 4], [1, 3, 4], [2], [2, 3, 4], [3, 4]],
 [[], [1], [1, 2, 3, 4], [1, 2, 4], [1, 3], [2, 3, 4], [2, 4], [3]],
 [[], [1], [1, 2, 3], [1, 2, 3, 4], [1, 4], [2, 3], [2, 3, 4], [4]],
 [[], [1, 2, 3, 4], [1, 2, 4], [1, 3, 4], [1, 4], [2], [2, 3], [3]],
 [[], [1, 2, 3], [1, 2, 3, 4], [1, 3], [1, 3, 4], [2], [2, 4], [4]],
 [[], [1, 2], [1, 2, 3], [1, 2, 3, 4], [1, 2, 4], [3], [3, 4], [4]],
 [[],
  [1],
  [1, 2],
  [1, 2, 3],
  [1, 2, 3, 4],
  [1, 2, 4],
  [1, 3],
  [1, 3, 4],
  [1, 4],
  [2],
  [2, 3],
  [2, 3, 4],
  [2, 4],
  [3],
  [3, 4],
  [4]]]